# Import Dependincies 

In [25]:
import pandas as pd
import spacy
import nltk
from collections.abc import Mapping
from experta import *

In [26]:
# import pandas as pd
# import spacy

# nlp = spacy.load("en_core_web_sm")

# def format_symptoms(symptoms):
#     """Replaces underscores with spaces in a string."""
#     return symptoms.replace('_', ' ')

# def preprocess_input(user_input):
#     """
#     Lowercases, formats, and preprocesses user input by lemmatizing,
#     removing stop words and POS tags that are likely verbs or verb-like in this context.
#     """
#     formatted_input = format_symptoms(user_input.lower())

#     doc = nlp(formatted_input)

#     remove_pos = ['PRON', 'ADP', 'AUX', 'CONJ', 'SCONJ', 'DET', 'PART', 'VERB']

#     processed_symptoms = [token.lemma_ for token in doc
#                           if token.is_alpha
#                           and not token.is_stop
#                           and token.pos_ not in remove_pos]

#     final_symptoms = [lemma for lemma in processed_symptoms if nlp(lemma)[0].pos_ not in ['VERB']]

#     return final_symptoms

# data = pd.read_csv('Medical Diagnosis Expert System.csv')

# df = data.copy()
# df['symptoms'] = df['symptoms'].apply(preprocess_input)

# df.head()



In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
import pandas as pd

nlp = spacy.load("en_core_web_sm")
df = pd.read_csv('Medical Diagnosis Expert System.csv')


symptom_phrases = set()
for symptoms in df['symptoms']:
    for symptom in symptoms.split(','):
        symptom_phrases.add(symptom.replace('_', ' ').strip())

matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
patterns = [nlp.make_doc(symptom) for symptom in symptom_phrases]
matcher.add("SYMPTOM", patterns)

def preprocess_input(user_input):
    """
    Extracts known symptom phrases from user input using PhraseMatcher.
    Returns a list of matched symptoms from the dataset.
    """
    formatted_input = user_input.lower().replace('_', ' ')
    doc = nlp(formatted_input)
    matches = matcher(doc)

    extracted_symptoms = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        extracted_symptoms.add(span.text.strip())

    return list(extracted_symptoms)

In [28]:
example_sentence = "I have a severe headache and feel very fatigued with a runny nose."
processed_example_no_verbs = preprocess_input(example_sentence)
print("\nExample Sentence:")
print(example_sentence)
print("Processed Symptoms (Lemmatized, No Verbs):")
print(processed_example_no_verbs)


Example Sentence:
I have a severe headache and feel very fatigued with a runny nose.
Processed Symptoms (Lemmatized, No Verbs):
['runny nose', 'headache']


In [29]:
# nlp = spacy.load("en_core_web_sm")

# data= pd.read_csv('Medical Diagnosis Expert System.csv')
# df =data.copy()
# df.head()

# def format_symptoms(symptoms):
#     return symptoms.replace('_', ' ')


# def preprocess_input(user_input):
#     formatted_input = format_symptoms(user_input.lower())
    
#     doc = nlp(formatted_input)
    
#     remove_pos = ['PRON', 'ADP', 'AUX', 'CONJ', 'SCONJ', 'DET', 'PART']
    
#     symptoms = [token.lemma_ for token in doc
#                 if token.is_alpha
#                 and not token.is_stop
#                 and token.pos_ not in remove_pos]
#     return symptoms
# df['symptoms'] = df['symptoms'].apply(preprocess_input)

# df.head()

In [30]:
print(type (df['disease'][0]))
print(type (df['symptoms'][0]))
print(type (df['precautions'][0]))

<class 'str'>
<class 'str'>
<class 'str'>


In [31]:
disease_data = {}

for _, row in df.iterrows():
    disease = row['disease']
    if isinstance(row['symptoms'], list):
        raw_symptoms = [s.strip() for s in row['symptoms']]
    else:
        raw_symptoms = [s.strip() for s in str(row['symptoms']).split(',')]
    processed_symptoms = set()
    for s in raw_symptoms:
        processed_symptoms.update(preprocess_input(s))

    precautions = [p.strip() for p in str(row['precautions']).split(',') if p]

    if disease not in disease_data:
        disease_data[disease] = {'symptoms': set(), 'precautions': set()}
    disease_data[disease]['symptoms'].update(processed_symptoms)
    disease_data[disease]['precautions'].update(precautions)

for disease in disease_data:
    disease_data[disease]['symptoms'] = sorted(disease_data[disease]['symptoms'])
    disease_data[disease]['precautions'] = sorted(disease_data[disease]['precautions'])


In [32]:
print(disease_data['Fungal infection']['symptoms'])
print(disease_data['Fungal infection']['precautions'])

['dischromic  patches', 'itching', 'nodal skin eruptions', 'skin rash']
['bath twice', 'keep infected area dry', 'use clean cloths', 'use detol or neem in bathing water']


In [33]:
print("Number of diseases:", len(disease_data))

Number of diseases: 41


In [34]:
from experta import KnowledgeEngine, Fact, Rule, Field

class Symptom(Fact):
    name = Field(str, mandatory=True)

class Diagnosis(Fact):
    disease = Field(str, mandatory=True)

class DiseaseDiagnosisEngine(KnowledgeEngine):
    def __init__(self, disease_data, input_symptoms):
        super().__init__()
        self.disease_data = disease_data
        self.input_symptoms = set(input_symptoms)
        self.possible_diseases = []
        self.matching_diseases = set()

    @Rule(Symptom(name=MATCH.symptom_name))
    def check_symptom(self, symptom_name):
        for disease, info in self.disease_data.items():
            if symptom_name in info['symptoms']:
                if not hasattr(self, f"matched_symptoms_{disease}"):
                    setattr(self, f"matched_symptoms_{disease}", {symptom_name})
                else:
                    getattr(self, f"matched_symptoms_{disease}").add(symptom_name)

    def get_diagnosis(self):
        self.matching_diseases = set()
        self.reset()
        for symptom in self.input_symptoms:
            self.declare(Symptom(name=symptom))
        self.run()

        for disease in self.disease_data:
            matched_symptoms = getattr(self, f"matched_symptoms_{disease}", set())
            if self.input_symptoms.issubset(matched_symptoms):
                self.matching_diseases.add(disease)
            if hasattr(self, f"matched_symptoms_{disease}"):
                delattr(self, f"matched_symptoms_{disease}")

        print("Diseases with all provided symptoms:", list(self.matching_diseases))
        return list(self.matching_diseases)

In [35]:
def get_precautions(disease_name):
    return disease_data.get(disease_name, {}).get('precautions', [])



In [38]:

def main():
    print("\nMedical Diagnosis Expert System")
    print("--------------------------------")
    print("Describe your symptoms (e.g., 'I have fever and headache'):")

    while True:
        user_input = input("\n> ").strip()

        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break

        if not user_input:
            print("Please enter your symptoms.")
            continue

        symptoms = preprocess_input(user_input)
  
        if not symptoms:
            print("No relevant symptoms detected. Please try again.")
            continue

        print(f"\n[Extracted Symptoms]: {', '.join(symptoms)}")

        print("\n[Diagnosis Results]")
        print("-------------------")


        engine = DiseaseDiagnosisEngine(disease_data, symptoms)
        engine.reset()
        diseases = engine.get_diagnosis()
        
        # print(f"Possible Diseases: {', '.join(diseases)}")

        if diseases:
            precautions = get_precautions(diseases[0])
            print(f"Diagnosis: {diseases[0]}")
            print("\nRecommended Precautions:")
            for i, precaution in enumerate(precautions, 1):
                print(f"{i}. {precaution}")
        else:
            print("No matching diseases found.")
            print("Please consult a healthcare professional.")
        
        print("\nEnter more symptoms or type 'exit' to quit.")


if __name__ == "__main__":
    main()


Medical Diagnosis Expert System
--------------------------------
Describe your symptoms (e.g., 'I have fever and headache'):

[Extracted Symptoms]: pus filled pimples, skin rash, scurring

[Diagnosis Results]
-------------------
Diseases with all provided symptoms: ['Acne']
Diagnosis: Acne

Recommended Precautions:
1. avoid fatty spicy food
2. avoid too many products
3. bath twice
4. drink plenty of water

Enter more symptoms or type 'exit' to quit.

[Extracted Symptoms]: itching, skin rash, dischromic  patches

[Diagnosis Results]
-------------------
Diseases with all provided symptoms: ['Fungal infection']
Diagnosis: Fungal infection

Recommended Precautions:
1. bath twice
2. keep infected area dry
3. use clean cloths
4. use detol or neem in bathing water

Enter more symptoms or type 'exit' to quit.

[Extracted Symptoms]: sunken eyes, vomiting, diarrhoea

[Diagnosis Results]
-------------------
Diseases with all provided symptoms: ['Gastroenteritis']
Diagnosis: Gastroenteritis

Reco